# #The Labeled Faces in the Wild face recognition dataset¶

In [4]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as pd
import torch
import numpy as np
from torch.nn import Parameter
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import pyro.ops.stats as stats
from sklearn import model_selection, datasets
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
assert pyro.__version__.startswith('1.6.0')
pyro.set_rng_seed(1)
import time as tm

In [2]:
#Load Faces dataset
faces = datasets.fetch_lfw_people() # 5828D, 13233 points

In [3]:
#Perform PCA on data and measure time elapsed
pca = PCA(n_components=50)
start = tm.time()
X_reduced = pca.fit_transform(faces.data)
end = tm.time()
print(end-start)
#Perform t-SNE on the reduced/original data and measure elapsed time
start = tm.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=45, n_iter=400)
reduced_digits_TSNE = tsne.fit_transform(faces.data)
end = tm.time()
print(end - start)

5.149921655654907
[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 13233 samples in 0.043s...
[t-SNE] Computed neighbors for 13233 samples in 20.707s...
[t-SNE] Computed conditional probabilities for sample 1000 / 13233
[t-SNE] Computed conditional probabilities for sample 2000 / 13233
[t-SNE] Computed conditional probabilities for sample 3000 / 13233
[t-SNE] Computed conditional probabilities for sample 4000 / 13233
[t-SNE] Computed conditional probabilities for sample 5000 / 13233
[t-SNE] Computed conditional probabilities for sample 6000 / 13233
[t-SNE] Computed conditional probabilities for sample 7000 / 13233
[t-SNE] Computed conditional probabilities for sample 8000 / 13233
[t-SNE] Computed conditional probabilities for sample 9000 / 13233
[t-SNE] Computed conditional probabilities for sample 10000 / 13233
[t-SNE] Computed conditional probabilities for sample 11000 / 13233
[t-SNE] Computed conditional probabilities for sample 12000 / 13233
[t-SNE] Computed conditional p

In [3]:
#GPLVM on faces dataset and elapsed time measure 
#Jupyter Notebook kernel needs to be restarted for testing with other parameters
y = torch.Tensor(faces.target.transpose())
X_prior_mean = torch.zeros(y.size(0), 2)  # shape: 13233 x 2
capture_time = y.new_tensor(faces.target)
time1 = capture_time
X_prior_mean[:, 0] = time1
Xu = stats.resample(X_prior_mean.clone(),16)
kernel = gp.kernels.RBF(input_dim=2, lengthscale=torch.ones(2))

# we clone here so that we don't change our prior during the course of training
X = Parameter(X_prior_mean.clone())
kernel = gp.kernels.RBF(input_dim=2, lengthscale=torch.ones(2))
gplvm1 = gp.models.SparseGPRegression(X, y, kernel,Xu,noise=torch.tensor(0.01), jitter=1e-5)
gplvm1.X = pyro.nn.PyroSample(dist.Normal(X, 0.01).to_event())
gplvm1.autoguide("X", dist.Normal)
start = tm.time()
losses = gp.util.train(gplvm1, num_steps=4000)
end = tm.time()
print(end - start)

160.3063690662384
